# --- Day 16: Reindeer Maze ---


In [14]:
# --- Part One ---

from icecream import ic
# import re
import time
from IPython.display import clear_output

filename = "input.txt"

#filename = "test1.txt" # decomment to use test
#filename = "test2.txt" # decomment to use test

COST_STEP = 1
COST_ROTATE = 1000
DIR = [">", "v", "<", "^"]
MOVE = { ">": (1, 0), "v": (0, 1), "<": (-1, 0), "^": (0, -1) }

class Maze:

    def __init__(self, filename = ""):

        self.walls = []
        self.paths = []
        self.start = None
        self.end = None
        self.size = (0,0)
        self.load_maze(filename)
        self.cost_map = {}
        self.exit_path = {}
        self.alltiles = set()

    def load_maze(self, filename):

        with open(filename, 'r') as file:
            grid  = [list(c) for c in [line.strip() for line in file]]
        self.size = (len(grid[0]), len(grid))

        for y, row in enumerate(grid):
            for x, obj in enumerate(row):
                if obj == "S":
                    self.start = (x,y)
                elif obj == "E":
                    self.end = (x,y)
                elif obj == "#":
                    self.walls.append((x,y))
                elif obj == ".":
                    self.paths.append((x,y))
                else:
                    pass    

    def __str__(self):
        retstr = ""
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) in self.walls:
                    retstr += "#"
                elif (x,y) in self.paths:
                    retstr += "."
                elif (x,y) == self.start:
                    retstr += "S"
                elif (x,y) == self.end:
                    retstr += "E"
                else:
                    pass
            retstr += "\n"
        return retstr
    
    def _left(self, pos, dir):
        new_dir = DIR[(DIR.index(dir)-1)%len(DIR)]
        return self._forward(pos, new_dir)

    def _right(self, pos, dir):
        new_dir = DIR[(DIR.index(dir)+1)%len(DIR)]
        return self._forward(pos, new_dir)

    def _forward(self,pos,dir):
        new_pos = (pos[0] + MOVE[dir][0], pos[1] + MOVE[dir][1])
        # print(f"{new_pos}")
        if new_pos in self.paths or new_pos == self.end or new_pos == self.start:
            return new_pos, dir
        return []

    def _new_path(self, pos, dir):
        retvalue = []
        ret = self._forward(pos, dir)
        # print (f"fwd: {ret}")
        if ret : retvalue.append(ret)
        ret = self._left(pos, dir)
        # print (f"lft: {ret}")
        if ret : retvalue.append(ret)
        ret = self._right(pos, dir)
        # print (f"rgt: {ret}")
        if ret : retvalue.append(ret)
        return retvalue


    def path_find(self, pos, dir, end = "E") -> dict:
        """
        Map costs for each position of the maze. 
        Keep only minimum costs. Returns the cost present on self.end position.
        """
        map_cost = {}
        map_cost[pos] = {"cost": 0, "dir": dir, "from":None}

        to_check = [(pos, dir)]

        while to_check: 
            p ,d = to_check.pop(0)
            if d != map_cost[p]["dir"]: continue # cell was reached from another direction
            # print (f"{p} {d}")
            for np, nd in self._new_path(p, d):
                nc = map_cost[p]["cost"]
                # calculate new cost
                if nd == d: #same direction
                    nc +=  COST_STEP
                else: 
                    nc += COST_STEP + COST_ROTATE
                c = map_cost.get(np,{"cost": None})["cost"]
                if not c or c > nc:
                    map_cost[np] = {"cost": nc, "dir": nd, "from":p}
                    to_check.append((np, nd))
        # get the exit path
        stp = self.end
        exit_path = {}
        while stp:
            c , d, f = map_cost[stp].values()
            if stp == self.end: d = "E"
            if stp == self.start: d = "S"
            exit_path[c] = (stp, d)
            stp = f
        
        self.exit_path = {k: v for k, v in sorted(exit_path.items())}
        self.cost_map = map_cost  
        return map_cost[self.end]["cost"] 

    def get_alltiles(self):
        """
        Get all the tiles for best paths
        """
        def _get_ck_values(pos, cost, dir):
            ck = []
            if dir == "^":
                np = (pos[0],pos[1]+1)
            if dir == "v":
                np = (pos[0],pos[1]-1)
            if dir == "<":
                np = (pos[0]+1,pos[1])            
            if dir == ">":
                np = (pos[0]-1,pos[1])
            ck.append((cost-COST_STEP, dir))
            if dir == "^" or dir == "v":
                ck.append((cost-COST_STEP-COST_ROTATE, "<"))
                ck.append((cost-COST_STEP-COST_ROTATE, ">"))
            if dir == "<" or dir == ">":
                ck.append((cost-COST_STEP-COST_ROTATE, "^"))
                ck.append((cost-COST_STEP-COST_ROTATE, "v"))
            return np, ck             
            
        tiles= set()
        to_check = []
        ck_data = {}
        p = self.end
        # print(self.cost_map[p])
        c , d, _ = self.cost_map[self.end].values()
        to_check.append(p)
        ck_data[p] = [(c, "^"), (c, ">")]
        while to_check:            
            p = to_check.pop(0)
            ck = ck_data[p]
            if p not in self.cost_map: continue
            if p in to_check: continue
            if self.cost_map[p]["cost"] not in [v[0] for v in ck]:
                continue
            tiles.add(p)
            # For each possible value in tyle check
            for c, d in ck:
                np, nck = _get_ck_values(p, c, d)
                to_check.append(np)
                ck_data[np] = nck
        self.alltiles = tiles
        return tiles
            
    def print_cost_map(self, cost_map = None):
        if not cost_map: cost_map = self.cost_map
        max_cost = max(c["cost"] for c in cost_map.values()) if cost_map else 0
        cell_w = len(f"[{max_cost}]")
        retstr = ""
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) in self.walls:
                    retstr += cell_w*"#"
                elif (x,y) in self.paths:
                    retstr += f"[{cost_map.get((x,y),{"cost": ''})["cost"]:^{cell_w-2}}]"
                elif (x,y) == self.start:
                    retstr += f"[{cost_map.get((x,y),{"cost": ''})["cost"]:^{cell_w-2}}]"
                elif (x,y) == self.end:
                    retstr += f"[{cost_map.get((x,y),{"cost": ''})["cost"]:^{cell_w-2}}]"
                else:
                    pass
            retstr += "\n"
        #clear_output()
        print(retstr)
        
    def print_exit_map(self, all=False, char = None, exit_path = None):
        if not exit_path: exit_path = self.exit_path
        retstr = ""
        if all == False:
            ex_path = {v[0]: v[1] for v in exit_path.values()}
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) in self.walls:
                    retstr += "#"
                elif all and (x,y) in self.alltiles:
                    retstr += char if char else "O"
                elif not all and (x,y) in ex_path:
                    retstr += char if char else ex_path[(x,y)] 
                else:
                    retstr += " "
            retstr += "\n"
        clear_output()
        print(retstr)



    
# ---- main ----

maze = Maze(filename)

# print(maze)

ret1 = maze.path_find(maze.start, ">")
print (f"The solution of for part 1 is: {ret1}")

map1 = maze.cost_map.copy()
#maze.print_cost_map()


maze.path_find(maze.end, "v")

map2 = maze.cost_map.copy()
#maze.print_cost_map()

map_sum = { k : {"cost": v["cost"] + map2[k]["cost"] , "dir":v["dir"], "from":v["from"]} for k, v in map1.items()}

#maze.print_cost_map(map_sum)

ret2 = len([k for k,v in map_sum.items() if v["cost"] in [ret1, ret1-1000,ret1+1000]])

#maze.print_tiles_map()



print (f"The solution of for part 2 is: {ret2}")

The solution of for part 1 is: 111480
The solution of for part 2 is: 527


In [17]:
# To solve use the count for part 2 but check the map to find few miscounted tiles.
exit_path = { i: (k, v["dir"]) for i, (k, v) in enumerate(map_sum.items()) if v["cost"] in [ret1, ret1-1000,ret1+1000] }
maze.print_exit_map(char = "█", exit_path= exit_path)

#############################################################################################################################################
#   #       #             #         #███████████████████#             #     #     #           #                █████████████████████#      █#
# ### ### ### ##### ####### ####### #█# ##### #########█##### ### # # ### # ### # ##### ##### # ########### ###█#█#█###█###█#######█# #####█#
#   # # # #   #   # #       #     # #█# #     #       #█#█████████  #   # #     # #     # #   #     #█████# #███████████████      #█#   # #█#
# # # # # # ### # # # ####### ### # #█# ### ### ##### #█#█# # ###█##### # ####### # ##### # #########█###█###█####### # # ### ### #█### # #█#
# # #   #   #   # #       #   # # # #█#   #   #   #   #███# #    █    #   #     #   #   # #   #███████# #█████#       #   # # #   #███#   #█#
# # ### ##### # ########### ### # # #█### ####### # ### ##### ###█### # ### ### ##### # # ### #█####### ####### # ### ##### # # ### #█### #█#
# #   